In [ ]:
import pandas as pd
import numpy as np
import requests

df_stream = pd.read_json('/Users/michellezhang/Documents/HMS/Jiam Lab/Spotify Project/Data/29/29_Spotify Extended Streaming History/Streaming_History_Audio_2013-2024_29.json')

df_stream['UniqueID'] = df_stream['master_metadata_album_artist_name'] + ":" + df_stream['master_metadata_track_name']

df_stream.head()

In [ ]:
#Checking URIs 

missing_uris = df_stream[df_stream['spotify_track_uri'].isnull()]
print("Rows with missing 'spotify_track_uri':")
print(missing_uris)

In [ ]:
#Cleaning df_stream by getting rid of rows without URIs

df_stream_cleaned = df_stream.dropna(subset=['spotify_track_uri'])

df_stream_cleaned.head()


In [ ]:

# create blank dictionary to store track URI, artist URI, and genres
feature_dict = {}

# convert track_uri column to an iterable list
track_uris = df_stream_cleaned['spotify_track_uri'].to_list()

num_entries = len(track_uris)

print("Number of entries in track_uris",num_entries)

# Removing duplicates
unique_track_uris = list(set(track_uris))

num_entries_unique = len(unique_track_uris)

print("Number of entries in track_uris",num_entries_unique)

print(unique_track_uris[:10])



In [ ]:
import spotipy
import time
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import random

client_id = '400cdcd09b0c49559616bee70d4a1afa'
client_secret = '55142e67c4b94f78adc0d7d808b791dc'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
sp.trace=False


In [ ]:

unique_track_uris

#put uri to dataframe
df = pd.DataFrame(unique_track_uris, columns= ['URI'])

#Initialize empty columns for audio features
features_list = ['energy', 'loudness', 'speechiness', 'valence', 'liveness', 'tempo', 'danceability']
for feature in features_list:
    df[feature] = ''

# Define a function to process a batch of URIs
def process_batch(batch_uris, df):
    for i, URI in enumerate(batch_uris):
        features = sp.audio_features(URI)
        if features and features[0]:
            df.loc[df['URI'] == URI, 'energy'] = features[0]['energy']
            df.loc[df['URI'] == URI, 'loudness'] = features[0]['loudness']
            df.loc[df['URI'] == URI, 'speechiness'] = features[0]['speechiness']
            df.loc[df['URI'] == URI, 'valence'] = features[0]['valence']
            df.loc[df['URI'] == URI, 'liveness'] = features[0]['liveness']
            df.loc[df['URI'] == URI, 'tempo'] = features[0]['tempo']
            df.loc[df['URI'] == URI, 'danceability'] = features[0]['danceability']

batch_size = 100
for start in range(0, len(unique_track_uris), batch_size):
    end = start + batch_size
    batch_uris = unique_track_uris[start:end]
    process_batch(batch_uris, df)
    print(f"Processed batch {start // batch_size + 1}")
    time.sleep(random.uniform(30, 60))  # Random delay between 30 to 60 seconds

